In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from pandas_profiling import ProfileReport
import numpy as np
from sklearn.linear_model import LinearRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
from joblib import Parallel, delayed
import time
import logging

In [2]:
class Linearregression():
    def __init__(self,file_path,predicted_col):
        self.file_path = file_path
        self.predicted_col = predicted_col
        logging.basicConfig(filename='linear_regression.log', level=logging.DEBUG,
                    format='%(asctime)s:%(levelname)s:%(message)s')
        logging.info('Linearregression class object is created.')
        
    def load_data(self):
        """
        Load csv file as pandas dataframe.
        
        
        Parameters
        ----------
        None
        
        Returns:
        ----------
        None        
        """
        logging.info('Dataset is getting loaded as pandas dataframe.')
        try:
            self.original_df = pd.read_csv(self.file_path)
        except FileNotFoundError:
            logging.error("File not found: exception occured while loading csv as pandas dataframe.")
        except pd.errors.EmptyDataError:
            logging.error("No data: exception occured while loading csv as pandas dataframe.")
        except pd.errors.ParserError:
            logging.errornt("Parse error: exception occured while loading csv as pandas dataframe.")
        except Exception as e:
            logging.error("{} occured while loading csv as pandas dataframe.".format(str(e)))
        
    def pandas_profiling(self,output_html):
        """
        Create pandas profiling report for the loaded dataset and
        save it as a html file.
        
        Parameters
        ----------
        output_html: Output htmla file named to be saved.
        
        Returns:
        ----------
        None
        """
        logging.info('Pandas profiling report is started.')
        pf = ProfileReport(self.original_df)
        pf.to_widgets()
        pf.to_file(output_html)
        logging.info('Pandas profiling report is finished ans saved inside {}.'.format(output_html))
        
    def check_NaN(self):
        """
        Calculate the number NaN values present in the dataset.
        
        Parameters
        ----------
        None.
        
        Returns:
        ----------
        None.
        """
        try:
            logging.info('Total number of NaN inside dataset is getting calculated.')
            return self.original_df.isna().sum().sum()
        except Exception as e:
            logging.error("{} occured while calculating total number of NaN inside dataset.".format(str(e)))
            return None
    
    def normalize(self):
        """
        Normalize each column of the dataset between 0 and 1 using min max normalization.
        
        Parameters
        ----------
        None.
        
        Returns:
        ----------
        None.
        """ 
        logging.info('Min Max normalization will be performed for all features inside dataset.')
        try:
            result = self.original_df.copy()
            ignore_columns = ["UDI", "Product ID", "Type"]
            for feature_name in self.original_df.columns:
                if feature_name not in ignore_columns :
                    max_value = self.original_df[feature_name].max()
                    min_value = self.original_df[feature_name].min()
                    result[feature_name] = (self.original_df[feature_name] - min_value) / (max_value - min_value)
                else:
                    result[feature_name] = self.original_df[feature_name]
            self.normalize_df = result
        except ZeroDivisionError:
            logging.error("Divison by zero: exception occured while min max normalization of dataframe.")
        except Exception as e:
            logging.error("{} occured while min max normalization of dataframe.".format(str(e)))
    def reverse_normalize(self, inp):
        """
        Do reverse normalization for the predicted value.
        
        Parameters
        ----------
        Inp: Input predicted value.
        
        Returns:
        ----------
        It returns the predicted value in the original range.
        """
        try:
            logging.info('Reverse of Min Max normalization will be performed for predicted feature value.')
            max_value = self.original_df[self.predicted_col].max()
            min_value = self.original_df[self.predicted_col].min()        
            return inp*(max_value - min_value)+min_value
        except Exception as e:
            logging.error("{} occured while reverse min max normalization of predicted feature value..".format(str(e)))
            return None
    
    def view_multicolinearity_by_vif(self):
        """
        This functions helps to judge the mulicolinearity among independent feature by calculating their
        VIF (Variable Inflation Factors).
        
        Parameters
        ----------
        None.
        
        Returns:
        ----------
        None.
        """
        logging.info('VIF values for all features inside dataset will be calculated.')
        try:
            ignore_columns = ["UDI", "Product ID", "Type"]
            X_variables_col = []
            for feature_name in self.normalize_df.columns:
                if feature_name not in ignore_columns:
                    X_variables_col.append(feature_name)
            self.X_variables = self.normalize_df[X_variables_col]
            self.vif_data = pd.DataFrame()
            self.vif_data["feature"] = self.X_variables.columns
            self.vif_data["VIF"] = [variance_inflation_factor(self.X_variables.values, i) for i in range(len(self.X_variables.columns))]
        except Exception as e:
            logging.error("{} occured while calculating VIF values for all features inside dataset.".format(str(e)))
        
    def drop_multicolinearity_by_vif(self, vif_thresh):
        """
        This functions drops tyhose columns whose values are more than threshold VIF passed as parameter.
        
        Parameters
        ----------
        vif_thresh: This is the threshold VIF value above which dataset column will be dropped.
        
        Returns:
        ----------
        None. 
        """
        logging.info('All features with VIF more than {} will be dropped from the dataset.'.format(vif_thresh))
        try:
            X = self.X_variables
            variables = [X.columns[i] for i in range(X.shape[1])]
            dropped=True
            while dropped:
                dropped=False
                vif = Parallel(n_jobs=-1,verbose=5)(delayed(variance_inflation_factor)(X[variables].values, ix) for ix in range(len(variables)))

                maxloc = vif.index(max(vif))
                if max(vif) > vif_thresh:
                    logging.info(time.ctime() + ' dropping \'' + X[variables].columns[maxloc] + '\' at index: ' + str(maxloc))
                    variables.pop(maxloc)
                    dropped=True

            logging.info('Remaining variables:')
            logging.info([variables])
            self.final_df = X[[i for i in variables]]
        except Exception as e:
            logging.error("{} occured while droping some of the feature from dataset based on vif threshold.".format(str(e)))
            
        
    def create_X_Y(self):
        """
        Create and reshuffle dataset based on Independent and dependent feature name.
        
        Parameters
        ----------
        None. 
        
        Returns:
        ----------
        None. 
        """
        logging.info('New dataset is created after reschuffle based on dependent feature')        
        try:
            self.Y = self.final_df[[self.predicted_col]]
            feature_name = self.final_df.columns.tolist()
            feature_name.remove(self.predicted_col)
            self.X = self.final_df[feature_name]
        except Exception as e:
            logging.error("{} occured while dataset reschuffle based on dependent feature.".format(str(e)))
    def build_model(self):
        """
        Build linear regression model.
        
        Parameters
        ----------
        None. 
        
        Returns:
        ----------
        None. 
        """
        logging.info('Linear regression model will be built now.')
        try:
            self.linear = LinearRegression()
            self.model = self.linear.fit(self.X,self.Y)
        except Exception as e:
            logging.error("{} occured while dbuilding linear regression model.".format(str(e)))
        
    def save_model(self,file_name):
        """
        Save the linear regresion model based on the input file name.
        
        Parameters
        ----------
        file_name: linear regression model will be saved with this file name. 
        
        Returns:
        ----------
        None. 
        """  
        logging.info('Save the linear regression model into file: {}.'.format(file_name))
        try:
            pickle.dump(self.model,open(file_name,'wb'))
        except Exception as e:
            logging.error("{} occured while saving linear regression model.".format(str(e)))            
        
    def calc_accuracy(self):
        """
        Calculate the accuracy of the linear regression model.
        
        Parameters
        ----------
        None. 
        
        Returns:
        ----------
        Returns the accuracy of the model. 
        """ 
        logging.info('Accuracy of the model will be calculated here.')
        try:
            return self.linear.score(self.X,self.Y)
        except Exception as e:
            logging.error("{} occured while calculating accuracy linear regression model.".format(str(e)))
            return None
    
    def predict(test_case):
        """
        Predict the dependent feature based on the input test case.
        
        Parameters
        ----------
        test_case: It is the independent variable list value. 
        
        Returns:
        ----------
        Returns the predicted feature. 
        """
        logging.info('Prediction will be done for the testcase {}.'.format(test_case))
        try:
            return self.linear.predict(test_case)
        except Exception as e:
            logging.error("{} occured while predicting dependent feature.".format(str(e)))
            return None        
        

In [3]:
linear_regr = Linearregression('challange_dataset.csv','Air temperature [K]')

#load_data()
linear_regr.load_data()

#profiling_data()
linear_regr.pandas_profiling('ori_df_profiling.html')

#fillna()
nan_count = linear_regr.check_NaN()
print(nan_count)

#normalize_data()
linear_regr.normalize()

#handle_multicolinearity()
linear_regr.view_multicolinearity_by_vif()
linear_regr.drop_multicolinearity_by_vif(vif_thresh=10)

#create independent feature and dependent feature
linear_regr.create_X_Y()

#build_model()
linear_regr.build_model()

#save_model()
linear_regr.save_model('linear_reg.sav')

#model_accuracy()
accuracy = linear_regr.calc_accuracy()
print(accuracy)

Summarize dataset:   0%|          | 0/27 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

0


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of  11 | elapsed:    0.5s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  11 out of  11 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed:    0.0s remaining:    0.0s


0.021227765262342824


[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    0.1s finished


In [4]:
linear_regr.X

Rotational speed [rpm]  Torque [Nm]  Tool wear [min]  TWF  HDF  PWF  \
0                   0.222934     0.535714         0.000000  0.0  0.0  0.0   
1                   0.139697     0.583791         0.011858  0.0  0.0  0.0   
2                   0.192084     0.626374         0.019763  0.0  0.0  0.0   
3                   0.154249     0.490385         0.027668  0.0  0.0  0.0   
4                   0.139697     0.497253         0.035573  0.0  0.0  0.0   
...                      ...          ...              ...  ...  ...  ...   
9995                0.253783     0.353022         0.055336  0.0  0.0  0.0   
9996                0.270081     0.384615         0.067194  0.0  0.0  0.0   
9997                0.277648     0.406593         0.086957  0.0  0.0  0.0   
9998                0.139697     0.614011         0.098814  0.0  0.0  0.0   
9999                0.193248     0.500000         0.118577  0.0  0.0  0.0   

      OSF  RNF  
0     0.0  0.0  
1     0.0  0.0  
2     0.0  0.0  
3     0.0  0.0  
4     0.0  0.0  
...   ...  ...  
9995  0.0  0.0  
9996  0.0  0.0  
9997  0.0  0.0  
9998  0.0  0.0  
9999  0.0  0.0  

[10000 rows x 8 columns]

In [5]:
linear_regr.Y

Air temperature [K]
0                0.304348
1                0.315217
2                0.304348
3                0.315217
4                0.315217
...                   ...
9995             0.380435
9996             0.391304
9997             0.402174
9998             0.402174
9999             0.402174

[10000 rows x 1 columns]

In [6]:
linear_regr.linear.intercept_

array([0.47396044])

In [7]:
linear_regr.linear.coef_

array([[ 0.10113843,  0.01460679,  0.01144753,  0.02791336,  0.29120109,
        -0.01219007, -0.00648137,  0.09181858]])

In [8]:
#predict_test_data()
testcase1 = [[0.222934,0.535714,0.000000,1.0,0.0,0.0,0.0,0.0]]
linear_regr.reverse_normalize(linear_regr.linear.predict(testcase1))

array([[300.19666365]])

In [9]:
testcase2 = [[0.222934,0.535714,0.000000,0.0,1.0,0.0,0.0,0.0]]
linear_regr.reverse_normalize(linear_regr.linear.predict(testcase2))

array([[302.61891082]])

In [10]:
testcase3 = [[0.639423,0.563973,0.000000,0.0,0.0,1.0,0.0,0.0]]
linear_regr.reverse_normalize(linear_regr.linear.predict(testcase3))

array([[300.21904164]])

In [11]:
testcase4 = [[0.639423,0.563973,0.000000,0.0,0.0,0.0,1.0,0.0]]
linear_regr.reverse_normalize(linear_regr.linear.predict(testcase4))

array([[300.27156167]])

In [12]:
testcase5 = [[0.639423,0.563973,0.000000,0.0,0.0,0.0,0.0,1.0]]
linear_regr.reverse_normalize(linear_regr.linear.predict(testcase5))

array([[301.17592126]])

In [13]:
testcase6 = [[0.639423,0.563973,0.000000,1.0,1.0,0.0,0.0,0.0]]
linear_regr.reverse_normalize(linear_regr.linear.predict(testcase6))

array([[303.26704323]])

In [14]:
testcase7 = [[0.639423,0.563973,0.456397,1.0,1.0,1.0,0.0,0.0]]
linear_regr.reverse_normalize(linear_regr.linear.predict(testcase7))

array([[303.20296107]])

In [15]:
testcase8 = [[0.639423,0.563973,0.456397,1.0,1.0,1.0,1.0,0.0]]
linear_regr.reverse_normalize(linear_regr.linear.predict(testcase8))

array([[303.14333245]])

In [16]:
testcase9 = [[0.639423,0.563973,0.456397,1.0,1.0,1.0,1.0,1.0]]
linear_regr.reverse_normalize(linear_regr.linear.predict(testcase9))

array([[303.98806343]])

In [17]:
testcase10 = [[0.674336,0.563973,0.538392,1.0,1.0,1.0,1.0,1.0]]
linear_regr.reverse_normalize(linear_regr.linear.predict(testcase10))

array([[304.02918454]])